In [32]:
from jupyterthemes import get_themes
import jupyterthemes as jt
from jupyterthemes.stylefx import set_nb_theme

set_nb_theme('chesterish')

In [2]:
import os
import urllib
import requests
import magic
import progressbar
from urllib.parse import quote
import random
from requests.exceptions import ReadTimeout
import pandas as pd
import time


C:\Users\drago\anaconda3\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\drago\anaconda3\lib\site-packages\numpy\.libs\libopenblas.PYQHXLVVQ7VESDPUVUADXEVJOBGHJPAY.gfortran-win_amd64.dll
C:\Users\drago\anaconda3\lib\site-packages\numpy\.libs\libopenblas.XWYDX2IKJW2NMTWSFYNGFUWKQU3LYTCZ.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


In [12]:
def check_webpage(url):
    checked_url = None
    try:
        request = requests.get(url, allow_redirects=True, timeout=10)
        if 'html' not in str(request.content):
            checked_url = request
    except ReadTimeout as err:
        print(err)
        pass
    return checked_url

check_webpage('https://www.google.com/search?q=perro+grande&rlz=1C1CHBF_esCL914CL914&sxsrf=ALiCzsaof6rKZfY9BUWZXdchYBv4JGUZkg:1668201940757&source=lnms&tbm=isch&sa=X&ved=2ahUKEwi6n97AiKf7AhV4q5UCHQCKDvcQ_AUoAXoECAIQAw&biw=1366&bih=625&dpr=1')

In [24]:
################
# ---> CONSTANTS
################

BASE_URL = 'https://www.google.com/search?q='
GOOGLE_PICTURE_ID = '''&biw=1536&bih=674&tbm=isch&sxsrf=ACYBGNSXXpS6YmAKUiLKKBs6xWb4uUY5gA:1581168823770&source=lnms&sa=X&ved=0ahUKEwioj8jwiMLnAhW9AhAIHbXTBMMQ_AUI3QUoAQ'''
HEADERS = {
    'User-Agent':
    "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/77.0.3865.90 Safari/537.36"
}
SCANNER_COUNTER = None


def generate_search_url(keywords):
    keywords_to_search = [str(item).strip() for item in keywords.split(',')]
    keywords_count = len(keywords_to_search)
    return keywords_to_search, keywords_count


def generate_urls(search):
    """Generates a URLS in the correct format that brings to Google Image seearch page"""
    return [(BASE_URL+quote(word)+GOOGLE_PICTURE_ID) for word in search]


def check_webpage(url):
    checked_url = None
    try:
        request = requests.get(url, headers = HEADERS,allow_redirects=True, timeout=10)
        if 'html' not in str(request.content):
            checked_url = request
    except (requests.exceptions.SSLError):
        pass
    return checked_url


def scan_webpage(webpage, extensions, timer):
    """Scans for pictures to download based on the keywords"""
    global SCANNER_COUNTER
    scanner = webpage.find
    found = False
    counter = 0
    while counter < timer:
        new_line = scanner('"https://', SCANNER_COUNTER + 1)  # How Many New lines
        SCANNER_COUNTER = scanner('"', new_line + 1)  # Ends of line
        buffor = scanner('\\', new_line + 1, SCANNER_COUNTER)
        if buffor != -1:
            object_raw = webpage[new_line + 1:buffor]
        else:
            object_raw = webpage[new_line + 1:SCANNER_COUNTER]
        if any(extension in object_raw for extension in extensions):
            found = True
            break
        counter += 1
    if found:
        object_ready = check_webpage(object_raw)
        return object_ready


class Downloader:
    """
        Main Downloader
        ::param extension:iterable of Files extensions
    """
    def __init__(self, extensions=None):
        if extensions:
            self._extensions = set(*[extensions])
        else:
            self._extensions = {'.jpg', '.png', '.ico', '.gif', '.jpeg'}
        self._directory = "simple_images/"
        self.get_dirs = set()
        self._cached_urls = {}

    @property
    def directory(self):
        return self._directory

    @directory.setter
    def directory(self, value):
        self._directory = value

    @property
    def cached_urls(self):
        return self._cached_urls

    @property
    def extensions(self):
        return self._extensions

    @extensions.setter
    def extensions(self, value):
        self._extensions = set([value])


    def get_urls(self):
        return [self._cached_urls[url][1].url
                for url in self._cached_urls]

    def _download_page(self, url):
        req = urllib.request.Request(url, headers=HEADERS)
        resp = urllib.request.urlopen(req)
        resp_data = str(resp.read())
        return resp_data

    def search_urls(self, keywords, limit=1, verbose=False, cache=True, timer=None):
        cache_out = {}
        search, count = generate_search_url(keywords)
        urls_ = generate_urls(search)
        timer = timer if timer else 1000
        max_progressbar = count * (list(range(limit+1))[-1]+1)
        bar = progressbar.ProgressBar(maxval=max_progressbar,
                                      widgets=[progressbar.Bar('=', '[', ']'), ' ', progressbar.Percentage()]).start()
        i = 0
        while i < count:
            global SCANNER_COUNTER
            SCANNER_COUNTER = -1
            url = urls_[i]
            path = self.generate_dir(search[i])
            raw_html = self._download_page(url) # Download the entire page from the google Picture search
            for _ in range(limit+1):
                webpage_url = scan_webpage(raw_html, self._extensions, timer)
                if webpage_url:
                    file_name = Downloader.gen_fn(webpage_url, search[i])
                    cache_out[file_name] = [path, webpage_url]
                    
                else:
                    pass
                bar.update(bar.currval + 1)
            i += 1
        bar.finish()
        if verbose:
            for url in cache_out:
                print(url)
        if cache:
            self._cached_urls = cache_out
        if not cache_out:
            print('==='*15 + ' < ' + 'NO PICTURES FOUND' + ' > ' + '==='*15)
        return cache_out
        
    def download(self, keywords=None, limit=1, verbose=False, cache=True, download_cache=False, timer=None):
        if not download_cache:
            content = self.search_urls(keywords, limit, verbose, cache, timer)
        else:
            content = self._cached_urls
            if not content:
                print('Downloader has not URLs saved in Memory yet, run Downloader.search_urls to find pics first')
#         print(content.items()[1][1])
        for name, (path, url) in content.items():
            with open(os.path.join(path, name), 'wb') as file:
                file.write(url.content)
            if verbose:
                print(f'File Name={name}, Downloaded from {url.url}')
            else:
                pass

    def _create_directories(self, name):
        dir_path = os.path.join(self._directory, name)
        try:
            if not os.path.exists(dir_path):
                os.makedirs(dir_path)
        except OSError:
            raise
        self.get_dirs.update([name])
        return

    def generate_dir(self, dir_name):
        """Generate Path and Directory, also check if Directory exists or not """
        dir_name = dir_name.replace(" ", "_")
        if dir_name in self.get_dirs:
            pass
        else:
            self._create_directories(dir_name)
        return self._directory + dir_name

    @staticmethod
    def gen_fn(check, name):
        """Create a file name string and generate a random identifiers otherwise won't import same pic twice"""
        id = str(hex(random.randrange(1000)))
        mime = magic.Magic(mime=True)
        file_type = mime.from_buffer(check.content)
        file_extension = f'.{file_type.split("/")[1]}'
        file_name = str(name) + "_" + id[2:] + file_extension
        return file_name

    def flush_cache(self):
        """Clear the Downloader instance cache"""
        self._cached_urls = set()


In [10]:
df = pd.read_csv('data_cact.csv')
df = df.drop(columns = ['Unnamed: 0'])
df1 = df[(df['rank'] == 'Species')].drop_duplicates()
df1

,kingdom,family,name,rank
0,Plantae,Aizoaceae,Tetragonia microcarpa,Species
1,Plantae,Cactaceae,Cereus horizontalis,Species
2,Plantae,Cactaceae,Copiapoa ahremephiana,Species
3,Plantae,Cactaceae,Copiapoa algarrobensis,Species
4,Plantae,Cactaceae,Copiapoa aphanes,Species
...,...,...,...,...
286,Plantae,Cactaceae,Pterocactus australis,Species
287,Plantae,Cactaceae,Pterocactus hickenii,Species
291,Plantae,Cactaceae,Maihuenia patagonica,Species
292,Plantae,Cactaceae,Austrocactus bertinii,Species


In [29]:
empty = []
  
# Traversing through Test
for root, dirs, files in os.walk('images_cact'):
  
    # Checking the size of tuple
    if not len(dirs) and len(files):
  
        # Adding the empty directory to
        # list
        empty.append(root)
  
# print("Empty Directories:")

plant_list = 'Cereus horizontalis'
for plant in empty:
    plant_list = plant_list + '|' + plant[12:].replace('_',' ') 
plant_list = plant_list[:-2]
len(empty)

148

In [34]:
names = list(df1['name'])
families = list(df1['family'])
len(df1['family'].unique())

df2 = df1[ df1['name'].str.contains(plant_list)== False]
names = list(df2['name'])
plant_list

'Cereus horizontalis|Airampoa soehrensii|Carpobrotus chilensis|Chiropetalum berteroanum|Chiropetalum canescens|Chiropetalum cremnophilum|Cistanthe sitiens|Colliguaja integerrima|Colliguaja odorifera|Copiapoa ahremephiana|Copiapoa algarrobensis|Copiapoa aphanes|Copiapoa atacamensis|Copiapoa australis|Copiapoa cinerascens|Copiapoa cinerea|Copiapoa coquimbana|Copiapoa corralensis|Copiapoa decorticans|Copiapoa desertorum|Copiapoa echinoides|Copiapoa esmeraldana|Copiapoa fiedleriana|Copiapoa fusca|Copiapoa gigantea|Copiapoa grandiflora|Copiapoa humilis|Copiapoa hypogaea|Copiapoa laui|Copiapoa leonensis|Copiapoa longispina|Copiapoa longistaminea|Copiapoa malletiana|Copiapoa marginata|Copiapoa megarhiza|Copiapoa mollicula|Copiapoa montana|Copiapoa pendulina|Copiapoa rupestris|Copiapoa schulziana|Copiapoa serpentisulcata|Copiapoa solaris|Copiapoa taltalensis|Copiapoa × scopa|Corryocactus brevistylus|Crassula connata|Croton chilensis|Croton colliguay|Cumulopuntia boliviana|Cumulopuntia glomerat

In [31]:
my_downloader = Downloader()

for i in range(len(names)):

    my_downloader.directory = 'C:/Users/drago/Documents/Plant_Reco/images_cact/'
    my_downloader.download(names[1:][i],limit=60, verbose=True)
    my_downloader.flush_cache()


[========================================================================] 100%


Mesembryanthemum nodiflorum_f5.png
Mesembryanthemum nodiflorum_2c1.png
Mesembryanthemum nodiflorum_38b.png
Mesembryanthemum nodiflorum_51.jpeg
Mesembryanthemum nodiflorum_70.jpeg
Mesembryanthemum nodiflorum_11f.jpeg
Mesembryanthemum nodiflorum_321.jpeg
Mesembryanthemum nodiflorum_227.jpeg
Mesembryanthemum nodiflorum_285.jpeg
Mesembryanthemum nodiflorum_337.jpeg
Mesembryanthemum nodiflorum_1d9.jpeg
Mesembryanthemum nodiflorum_27a.png
Mesembryanthemum nodiflorum_37.jpeg
Mesembryanthemum nodiflorum_1d8.jpeg
Mesembryanthemum nodiflorum_7d.jpeg
Mesembryanthemum nodiflorum_245.jpeg
Mesembryanthemum nodiflorum_2c3.jpeg
Mesembryanthemum nodiflorum_3e3.jpeg
Mesembryanthemum nodiflorum_f.jpeg
Mesembryanthemum nodiflorum_39.jpeg
Mesembryanthemum nodiflorum_5a.jpeg
Mesembryanthemum nodiflorum_2b4.jpeg
Mesembryanthemum nodiflorum_3c9.jpeg
Mesembryanthemum nodiflorum_113.jpeg
Mesembryanthemum nodiflorum_6d.jpeg
Mesembryanthemum nodiflorum_27c.jpeg
Mesembryanthemum nodiflorum_353.jpeg
Mesembryanthemu

[========================================================================] 100%


Tetragonia tetragonoides_20a.png
Tetragonia tetragonoides_3cc.png
Tetragonia tetragonoides_36d.png
Tetragonia tetragonoides_70.jpeg
Tetragonia tetragonoides_1d2.jpeg
Tetragonia tetragonoides_e7.jpeg
Tetragonia tetragonoides_c.jpeg
Tetragonia tetragonoides_12e.jpeg
Tetragonia tetragonoides_2c4.jpeg
Tetragonia tetragonoides_112.jpeg
Tetragonia tetragonoides_169.jpeg
Tetragonia tetragonoides_119.jpeg
Tetragonia tetragonoides_8e.jpeg
Tetragonia tetragonoides_26c.jpeg
Tetragonia tetragonoides_2d7.jpeg
Tetragonia tetragonoides_1af.jpeg
Tetragonia tetragonoides_1dc.jpeg
Tetragonia tetragonoides_52.jpeg
Tetragonia tetragonoides_3ca.jpeg
Tetragonia tetragonoides_39e.jpeg
Tetragonia tetragonoides_1fb.jpeg
Tetragonia tetragonoides_30.jpeg
Tetragonia tetragonoides_53.jpeg
Tetragonia tetragonoides_223.jpeg
Tetragonia tetragonoides_59.jpeg
Tetragonia tetragonoides_146.jpeg
Tetragonia tetragonoides_1c1.jpeg
Tetragonia tetragonoides_19f.jpeg
Tetragonia tetragonoides_27e.jpeg
Tetragonia tetragonoides_3

[========================================================================] 100%


Euphorbia hypericifolia_390.png
Euphorbia hypericifolia_5f.png
Euphorbia hypericifolia_360.png
Euphorbia hypericifolia_34c.png
Euphorbia hypericifolia_7a.jpeg
Euphorbia hypericifolia_1ad.jpeg
Euphorbia hypericifolia_326.jpeg
Euphorbia hypericifolia_e1.jpeg
Euphorbia hypericifolia_a3.jpeg
Euphorbia hypericifolia_273.jpeg
Euphorbia hypericifolia_244.jpeg
Euphorbia hypericifolia_da.jpeg
Euphorbia hypericifolia_23a.jpeg
Euphorbia hypericifolia_101.jpeg
Euphorbia hypericifolia_af.jpeg
Euphorbia hypericifolia_26a.jpeg
Euphorbia hypericifolia_11f.jpeg
Euphorbia hypericifolia_217.jpeg
Euphorbia hypericifolia_2b6.jpeg
Euphorbia hypericifolia_a8.jpeg
Euphorbia hypericifolia_a5.jpeg
Euphorbia hypericifolia_bb.png
Euphorbia hypericifolia_33b.jpeg
Euphorbia hypericifolia_1d.jpeg
Euphorbia hypericifolia_1d5.jpeg
Euphorbia hypericifolia_125.jpeg
Euphorbia hypericifolia_2b.jpeg
Euphorbia hypericifolia_2eb.jpeg
Euphorbia hypericifolia_3a4.jpeg
Euphorbia hypericifolia_3c6.jpeg
Euphorbia hypericifolia_38

ReadTimeout: HTTPSConnectionPool(host='inta.gob.ar', port=443): Read timed out. (read timeout=10)

In [153]:
my_downloader = Downloader()

for path in empty:
    
    time.sleep(5)
    my_downloader.directory = 'C:/Users/drago/Documents/Plant_Reco/images/' + path.split('\\')[1] + '/'
    my_downloader.download(path.split('\\')[2].replace('_',' '),limit=20, verbose=True)
    time.sleep(5)
    my_downloader.flush_cache()
    

KeyboardInterrupt: 